In [1]:
# import modules
import requests
import pandas as pd
import csv
import xml.etree.ElementTree as ET
import datetime
from datetime import datetime

In [2]:
# set variables

# Replace these with your actual Plex server's IP address and Plex token
plexAddress = '69.160.116.115'
plexToken = 'P6wzVcNKJtB2gUxrnbj3'
serverToken = 'L-BMppkF6CmHCextDgLs'

#CSV where plex data will be outputted 
csvFile = r'E:\Projects\Plex\plex_play_history.csv'


# The URL with the auth token to access the Plex API from the server 
# https://www.plexopedia.com/plex-media-server/general/plex-token/#plexservertoken

serverToken = f'https://plex.tv/api/v2/server/access_tokens?auth_token={serverToken}'
deviceInfo = f'http://{plexAddress}:32400/devices/?X-Plex-Token={plexToken}'
serverHistory = f'http://{plexAddress}:32400/status/sessions/history/all?X-Plex-Token={plexToken}'


In [3]:
#Functions 

###########################################################################################

# XML data from Plex API
def serverXML(url):
    response = requests.get(url)
    if response.status_code == 200:
        # print(response.text)
        return response.text
    else:
        raise Exception(f"Failed to retrieve data: {response.status_code}")

###########################################################################################

# Parse XML and extract user aliases
def parseXML(xml_data):
    root = ET.fromstring(xml_data)

    # Dictionary to hold user titles (aliases)
    user_aliases = {}

    # Extract user information
    for token in root.findall(".//token[@type='server']/invited"):
        user_id = token.get('id')
        user_title = token.get('title')
        user_aliases[user_id] = user_title

    return user_aliases

###########################################################################################
# Device information from Plex API and parse it
def parseDeviceInfo(plexAddress, plexToken):
    response = requests.get(deviceInfo)

    if response.status_code == 200:
        root = ET.fromstring(response.text)
        device_info  = {}

        for device in root.findall('.//Device'):
            device_id = device.get('id')
            device_name = device.get('name', f'Unknown_Device_{device_id}')
            platform = device.get('platform', 'Unknown')
            device_info[device_id] = {'device_name': device_name, 'platform': platform}

        return device_info
    else:
        raise Exception(f"Failed to retrieve device information: {response.status_code}")

###########################################################################################

# Func-ception ///Embedded in plexHistory function using the rating key to extract Duration and Genre
def fetchMediaDetails(plexAddress, plexToken, ratingKey):
    url = f"http://{plexAddress}:32400/library/metadata/{ratingKey}?X-Plex-Token={plexToken}"
    response = requests.get(url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        duration = root.find('.//Media').get('duration')
        first_genre_element = root.find('.//Genre')
        first_genre = first_genre_element.get('tag') if first_genre_element is not None else 'Unknown'

        return duration, first_genre
    else:
        raise Exception(f"Failed to retrieve media details: {response.status_code}")
    
###########################################################################################

# Fetch play history and write to CSV
def plexHistory(plexAddress, plexToken, device_info, user_aliases):
    response = requests.get(serverHistory)
    if response.status_code == 200:
        root = ET.fromstring(response.content)

        with open(csvFile, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Title', 'Series Title', 'Type', 'Index', 'Duration', 'Genre', 'Originally Available', 'Viewed At', 'User Alias', 'User ID', 'Device Alias', 'Device ID', 'Platform'])

            for video in root.findall('.//Video'):
                title = video.get('title', 'Unknown')
                series_title = video.get('grandparentTitle', 'N/A')
                content_type = video.get('type', 'Unknown')
                episode_or_movie_index = video.get('index', 'Unknown')
                ratingKey = video.get('ratingKey')
                if ratingKey:
                    duration_ms,first_genre = fetchMediaDetails(plexAddress, plexToken, ratingKey) 
                else:
                    duration_ms = 'Unknown'
                    first_genre = 'Unknown'
                orig_avail = video.get('originallyAvailableAt', 'Unknown')
                viewed_at_epoch = video.get('viewedAt', 'Unknown')
                account_id = video.get('accountID', 'Unknown')
                device_id = video.get('deviceID', 'Unknown')
                viewed_at = datetime.fromtimestamp(int(viewed_at_epoch)).strftime('%Y-%m-%d %H:%M:%S') if viewed_at_epoch.isdigit() else 'Unknown'
                user_alias = user_aliases.get(account_id, 'Unknown')
                device_info_entry = device_info.get(device_id, {'device_name': 'Unknown', 'platform': 'Unknown'})
                device_alias = device_info_entry['device_name']
                platform = device_info_entry['platform']

                writer.writerow([title, series_title, content_type, episode_or_movie_index, duration_ms,first_genre, orig_avail, viewed_at, user_alias, account_id, device_alias, device_id, platform])

        print(f'Data written to {csvFile}')
        return csvFile
    else:
        print(f'Failed to retrieve data: {response.status_code}')
        return None


In [4]:
# Run functions 

# Fetch the XML data for aliases
xml = serverXML(serverToken)

# Parse the XML data and get aliases
userData = parseXML(xml)

# Fetch and parse device information from Plex API
deviceData = parseDeviceInfo(plexAddress, plexToken)

# Now fetch the play history and write to CSV
plexHistory(plexAddress, plexToken, deviceData, userData) #Dependent on activity of server, for ~1800 rows took 10mins

Data written to E:\Projects\Plex\plex_play_history.csv


'E:\\Projects\\Plex\\plex_play_history.csv'

In [5]:
#Save dataframe to csv
plexDF = pd.read_csv(csvFile)

In [6]:
#Check for number of rows before data cleaning
plexDF.shape

(1972, 13)

In [7]:
# Let's look at some sample data
plexDF.head()

,Title,Series Title,Type,Index,Duration,Genre,Originally Available,Viewed At,User Alias,User ID,Device Alias,Device ID,Platform
0,Changes,The Man Who Fell to Earth,episode,6,3408266,Unknown,2022-06-05,2022-07-07 14:29:05,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx
1,Unwashed and Somewhat Slightly Dazed,The Man Who Fell to Earth,episode,2,3323306,Unknown,2022-05-01,2022-07-07 14:50:08,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx
2,Hallo Spaceboy,The Man Who Fell to Earth,episode,1,3284714,Unknown,2022-04-24,2022-07-07 14:50:13,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx
3,Hallo Spaceboy,The Man Who Fell to Earth,episode,1,3284714,Unknown,2022-04-24,2022-07-07 14:50:27,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx
4,New Angels of Promise,The Man Who Fell to Earth,episode,3,3431529,Unknown,2022-05-08,2022-07-07 14:50:27,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx


In [8]:
# Now let's look at the Data types
plexDF.dtypes

Title                   object
Series Title            object
Type                    object
Index                   object
Duration                object
Genre                   object
Originally Available    object
Viewed At               object
User Alias              object
User ID                  int64
Device Alias            object
Device ID                int64
Platform                object
dtype: object

In [9]:
# Convert the date columns from object to datetime
plexDF['Originally Available'] = pd.to_datetime(plexDF['Originally Available'], errors='coerce')

plexDF['Viewed At'] = pd.to_datetime(plexDF['Viewed At'], errors='coerce')

plexDF['Viewed At'] = plexDF['Viewed At'].dt.date #removes the time from the date

In [10]:
#Data cleaning for Duration

plexDF['Duration'] = plexDF['Duration'].replace('Unknown', 0)   #replace Unknown values with 0


plexDF['Duration'] = plexDF['Duration'].fillna(0).astype(int)   #Updates the column type to integer



# Calculate average durations dependent on the type of media, being a show or movie

avg_duration_episode = plexDF[(plexDF['Type'] == 'episode') & (plexDF['Duration'] > 0)]['Duration'].mean()
avg_duration_movie = plexDF[(plexDF['Type'] == 'movie') & (plexDF['Duration'] > 0)]['Duration'].mean()

# Update Unknown/0 values with the average
plexDF.loc[(plexDF['Type'] == 'episode') & (plexDF['Duration'].isna() | (plexDF['Duration'] == 0)), 'Duration'] = avg_duration_episode
plexDF.loc[(plexDF['Type'] == 'movie') & (plexDF['Duration'].isna() | (plexDF['Duration'] == 0)), 'Duration'] = avg_duration_movie


plexDF['Duration'] = plexDF['Duration'].astype(int) #Just a double check to ensure the data type is correct


C:\Users\nehrou\AppData\Local\Temp\ipykernel_1084\275340796.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2690669.872468528' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  plexDF.loc[(plexDF['Type'] == 'episode') & (plexDF['Duration'].isna() | (plexDF['Duration'] == 0)), 'Duration'] = avg_duration_episode


In [11]:
# Duration is in milliseconds, so to make the data more accessible we're goign to convert it to minutes and hours
plexDF['Duration_Mins'] = (plexDF['Duration']/60000).round(2) 
plexDF['Duration_Hrs'] = (plexDF['Duration_Mins']/60).round(1) 

In [12]:
#Let's check how the dataframe looks like now
plexDF.head(5)

,Title,Series Title,Type,Index,Duration,Genre,Originally Available,Viewed At,User Alias,User ID,Device Alias,Device ID,Platform,Duration_Mins,Duration_Hrs
0,Changes,The Man Who Fell to Earth,episode,6,3408266,Unknown,2022-06-05,2022-07-07,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx,56.80,0.9
1,Unwashed and Somewhat Slightly Dazed,The Man Who Fell to Earth,episode,2,3323306,Unknown,2022-05-01,2022-07-07,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx,55.39,0.9
2,Hallo Spaceboy,The Man Who Fell to Earth,episode,1,3284714,Unknown,2022-04-24,2022-07-07,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx,54.75,0.9
3,Hallo Spaceboy,The Man Who Fell to Earth,episode,1,3284714,Unknown,2022-04-24,2022-07-07,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx,54.75,0.9
4,New Angels of Promise,The Man Who Fell to Earth,episode,3,3431529,Unknown,2022-05-08,2022-07-07,Epy0n,149436557,Julians-MacBook-Pro.local,6,osx,57.19,1.0


In [13]:
# Noticed using the API that there were duplicated rows of data, so we have to clean that up for accurate results

plexMovies = plexDF['Type'] == 'movie'

plexShows = plexDF['Type'] == 'episode'

plexDF.loc[plexMovies,:] = plexDF.loc[plexMovies].drop_duplicates(subset=['Title', 'Type', 'User ID', 'Viewed At'], keep='first')
#moviesDF = plexDF.loc[plexMovies].drop_duplicates(subset=['Title', 'Type', 'User ID', 'Viewed At'], keep='first')

plexDF.loc[plexShows,:] = plexDF.loc[plexShows].drop_duplicates(subset=['Title','Series Title','Index','User ID','Viewed At','Originally Available'], keep='first')

plexDF = plexDF.dropna(how='all')

plexDF.shape #Let's check how many rows we have now

(1904, 15)

In [14]:
# Want to standardise the 
counts = plexDF[['Device Alias']].value_counts()
counts = counts.reset_index() 
counts.columns = ['Alias', 'Count']

countsPath = r'E:\Projects\Plex\plex_counts.csv' #Path to save csv, just to go over the data and standardize it to our liking first

counts.to_csv(countsPath)

In [15]:
# After we standardize the data in the csv, we'll import it to dataframe and use it as a control table to 
controlDF = pd.read_csv(countsPath)

controlDF

,Unnamed: 0,Alias,Device,Count
0,0,LG 43UN7300PUF,TV,752
1,1,iPhone,Phone,302
2,2,Chromecast Google TV,Streaming Stick,196
3,3,TCLRoku TV - X00000UMWJH6,TV,180
4,4,Bee-Pro,Laptop,144
5,5,Chrome,Browser,87
6,6,Galaxy Tab S6 Lite,Tablet,48
7,7,HIVE,Laptop,47
8,8,Julians-MacBook-Pro.local,Laptop,27
9,9,iPad,Tablet,25


In [16]:
plexDF = plexDF.merge(controlDF[['Alias','Device']], left_on='Device Alias',right_on='Alias', how='left')


plexDF.head(10)

,Title,Series Title,Type,Index,Duration,Genre,Originally Available,Viewed At,User Alias,User ID,Device Alias,Device ID,Platform,Duration_Mins,Duration_Hrs,Alias,Device
0,Changes,The Man Who Fell to Earth,episode,6,3408266.0,Unknown,2022-06-05,2022-07-07,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,56.80,0.9,Julians-MacBook-Pro.local,Laptop
1,Unwashed and Somewhat Slightly Dazed,The Man Who Fell to Earth,episode,2,3323306.0,Unknown,2022-05-01,2022-07-07,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,55.39,0.9,Julians-MacBook-Pro.local,Laptop
2,Hallo Spaceboy,The Man Who Fell to Earth,episode,1,3284714.0,Unknown,2022-04-24,2022-07-07,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,54.75,0.9,Julians-MacBook-Pro.local,Laptop
3,New Angels of Promise,The Man Who Fell to Earth,episode,3,3431529.0,Unknown,2022-05-08,2022-07-07,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,57.19,1.0,Julians-MacBook-Pro.local,Laptop
4,Moonage Daydream,The Man Who Fell to Earth,episode,5,2851721.0,Unknown,2022-05-29,2022-07-07,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,47.53,0.8,Julians-MacBook-Pro.local,Laptop
5,Under Pressure,The Man Who Fell to Earth,episode,4,3383881.0,Unknown,2022-05-15,2022-07-07,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,56.40,0.9,Julians-MacBook-Pro.local,Laptop
6,Like Crazy,NaN,movie,Unknown,5402621.0,Drama,2012-01-27,2022-07-07,Unknown,1.0,Chrome,2.0,Chrome,90.04,1.5,Chrome,Browser
7,Like Crazy,NaN,movie,Unknown,5402621.0,Drama,2012-01-27,2022-07-07,Tia,153388634.0,Chrome,7.0,Chrome,90.04,1.5,Chrome,Browser
8,Cracked Actor,The Man Who Fell to Earth,episode,7,3432297.0,Unknown,2022-06-12,2022-07-08,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,57.20,1.0,Julians-MacBook-Pro.local,Laptop
9,The Pretty Things Are Going To Hell,The Man Who Fell to Earth,episode,8,3562057.0,Unknown,2022-06-19,2022-07-08,Epy0n,149436557.0,Julians-MacBook-Pro.local,6.0,osx,59.37,1.0,Julians-MacBook-Pro.local,Laptop


In [17]:
#Let's remove the columns we do not require
plexDF.drop(columns=['Alias','Device ID','User ID','Duration'], inplace=True)

In [18]:
plexDF['User Alias'].value_counts()

User Alias
Epy0n        660
Unknown      649
Tia          402
Kashae       160
Christine     25
Greg           6
Pachriis       2
Name: count, dtype: int64

In [19]:
plexDF['Device Alias'].replace(to_replace='Julians-MacBook-Pro.local', value='MacBook', inplace=True)
plexDF['Device Alias'].replace(to_replace='Tia’s iPhone', value='iPhone', inplace=True)
plexDF['User Alias'].replace(to_replace='Unknown', value='nehrou', inplace=True)
plexDF['User Alias'].replace(to_replace='Tia', value='PinkChick', inplace=True)
plexDF['User Alias'].replace(to_replace='Kashae', value='nadia', inplace=True)
plexDF['User Alias'].replace(to_replace='Christine', value='Chrissy', inplace=True)
plexDF['User Alias'].replace(to_replace='Greg', value='5Head', inplace=True)
plexDF['User Alias'].replace(to_replace='Epy0n', value='ASC', inplace=True)
plexDF['User Alias'].replace(to_replace='Pachriis', value='Chocolate', inplace=True)


In [20]:
# Here's our final dataframe
plexDF.head(10)

,Title,Series Title,Type,Index,Genre,Originally Available,Viewed At,User Alias,Device Alias,Platform,Duration_Mins,Duration_Hrs,Device
0,Changes,The Man Who Fell to Earth,episode,6,Unknown,2022-06-05,2022-07-07,ASC,MacBook,osx,56.80,0.9,Laptop
1,Unwashed and Somewhat Slightly Dazed,The Man Who Fell to Earth,episode,2,Unknown,2022-05-01,2022-07-07,ASC,MacBook,osx,55.39,0.9,Laptop
2,Hallo Spaceboy,The Man Who Fell to Earth,episode,1,Unknown,2022-04-24,2022-07-07,ASC,MacBook,osx,54.75,0.9,Laptop
3,New Angels of Promise,The Man Who Fell to Earth,episode,3,Unknown,2022-05-08,2022-07-07,ASC,MacBook,osx,57.19,1.0,Laptop
4,Moonage Daydream,The Man Who Fell to Earth,episode,5,Unknown,2022-05-29,2022-07-07,ASC,MacBook,osx,47.53,0.8,Laptop
5,Under Pressure,The Man Who Fell to Earth,episode,4,Unknown,2022-05-15,2022-07-07,ASC,MacBook,osx,56.40,0.9,Laptop
6,Like Crazy,NaN,movie,Unknown,Drama,2012-01-27,2022-07-07,nehrou,Chrome,Chrome,90.04,1.5,Browser
7,Like Crazy,NaN,movie,Unknown,Drama,2012-01-27,2022-07-07,PinkChick,Chrome,Chrome,90.04,1.5,Browser
8,Cracked Actor,The Man Who Fell to Earth,episode,7,Unknown,2022-06-12,2022-07-08,ASC,MacBook,osx,57.20,1.0,Laptop
9,The Pretty Things Are Going To Hell,The Man Who Fell to Earth,episode,8,Unknown,2022-06-19,2022-07-08,ASC,MacBook,osx,59.37,1.0,Laptop


In [21]:
# Save csv to do some data visualization 
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
finalCSV = r'E:\Projects\Plex\{}_plexHistory.csv'.format(current_date)

plexDF.to_csv(finalCSV,index=False)